In this work, you will use the APIs of PyG and DGL to implement some basic functions.

In [4]:
pip install ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.4 kB 325.1 kB/s eta 0:00:01
   ----------- --------------------------- 41.0/139.4 kB 326.8 kB/s eta 0:00:01
   ------------------------- ------------- 92.2/139.4 kB 525.1 kB/s eta 0:00:01
   ------------------------- ------------- 92.2/139.4 kB 525.1 kB/s eta 0:00:01
   ------------------------------ ------- 112.6/139.4 kB 409.6 kB/s eta 0:00:01
   -------------------------------------- 139.4/139.4 kB 485.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.0 kB ? eta -:--:--
   ----- --------------------------------- 30.7/215.0 kB 435.7 kB/s eta 0:00:01
   ----------- --------------------------- 61.4/215.0 kB 550.5 kB/s eta 0:00:01
   ---------------- ---------------------- 92.2/215.0 kB 585.1 kB/s eta 0:

You need to run the following commands to install the GNN libraries (Only CPU version).

The most popular GNN models can be written as follows:

$$
h_i^{(l+1)}=\sigma(b^{(l)}+\sum_{j\in\mathcal{N}(i)}e_{ij}h_j^{(l)}W^{(l)})
$$

where $h_i^{(l+1)}$ is the output feature, $\sigma$ is the activation function, $e_{ij}$ is the edge weight, $W^{(l)}$ is the learnable parameters, $b^{(l)
}$ is the bias.

First, you will use the PyTorch-Geometric(PyG) to implement this convolution layer.

In [2]:
import torch
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing

class PyG_conv(MessagePassing):
    def __init__(self, in_channel, out_channel):
        super(PyG_conv, self).__init__(aggr='add')
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.W = nn.Parameter(torch.ones((in_channel, out_channel)))
        self.b = nn.Parameter(torch.ones(out_channel))

    def forward(self, x, edge_index, edge_weight):
        return self.propagate(edge_index, x=x, edge_weight=edge_weight)

    def message(self, x_j, edge_weight):
        return edge_weight.view(-1,1) * x_j

    def update(self, aggr_out):
        return aggr_out @ self.W + self.b

E:\conda\miniconda\envs\PyG\lib\site-packages\torch_geometric\transforms\gdc.py:13: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return numba.jit(cache=True)(func)


In [3]:
import numpy as np
edge_index = torch.tensor([[0,1,1,2,2,4],[2,0,2,3,4,3]])
x = torch.ones((5, 8))
edge_weight = 2 * torch.ones(6)  # 调整边权重的数量与边的数量一致
conv = PyG_conv(8, 4)
output = conv(x, edge_index, edge_weight)
print(output)
print(output.shape)

assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                      [ 1.,  1.,  1.,  1.],
                      [33., 33., 33., 33.],
                      [33., 33., 33., 33.],
                      [17., 17., 17., 17.]])


tensor([[17., 17., 17., 17.],
        [ 1.,  1.,  1.,  1.],
        [33., 33., 33., 33.],
        [33., 33., 33., 33.],
        [17., 17., 17., 17.]], grad_fn=<AddBackward0>)
torch.Size([5, 4])


You may run the following code to check the correctness.

Now, you will implement the same functions with DGL.

In [17]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn

class DGL_conv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(DGL_conv, self).__init__()
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.W = nn.Parameter(torch.ones(in_channel, out_channel))
        self.b = nn.Parameter(torch.ones(out_channel))
        
    def forward(self, g, h, e):
        g.ndata['h'] = h
        g.edata['edge_weight'] = edge_weight
        g.ndata['hm'] = g.ndata['h'] @ self.W
        g.update_all(fn.u_mul_e('hm', 'edge_weight', 'm'), fn.sum('m', 'h'))
        g.ndata['h'] = g.ndata['h'] + self.b
        return g.ndata['h']

Also, you can also run the code below to check the correctness.

In [18]:
src = torch.tensor([0, 1, 1, 2, 2, 4])
dst = torch.tensor([2, 0, 2, 3, 4, 3])
h = torch.ones((5, 8))
g = dgl.graph((src, dst))
edge_weight = 2 * torch.ones(6)
conv = DGL_conv(8, 4)
output = conv(g, h, edge_weight)
print(output)
import numpy as np
assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                      [ 1.,  1.,  1.,  1.],
                      [33., 33., 33., 33.],
                      [33., 33., 33., 33.],
                      [17., 17., 17., 17.]])

tensor([[17., 17., 17., 17.],
        [ 1.,  1.,  1.,  1.],
        [33., 33., 33., 33.],
        [33., 33., 33., 33.],
        [17., 17., 17., 17.]], grad_fn=<AddBackward0>)
